In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np


In [2]:
def get_tecs(raw_dir = "stock_data/"):
    data_gen = DataGenerator(random=False, base_dir = raw_dir)
    data_gen.rewind()
    stateUtil = StateUtil(data_gen)
    data = []
    data_count = data_gen.max_steps()
    tec = TecAn()
    for i in tqdm(range(data_count)):
        states = data_gen.next()
        for raw in states:
            price = raw[stateUtil.PRICE_KEY]
            amount = raw[stateUtil.AMOUNT_KEY]
            data.append([price, amount])


    def calcule_tecs():
        tecs = []
        df = pd.DataFrame(data, columns = ['Close', 'Volume'])
        close = df['Close']
        volume = df['Volume']
        import math
        for ta in tec.tas:
            value = ta(close, volume, 0.0, 0.0)
            tecs.append(value)
        return tecs
    
    return calcule_tecs()


In [3]:
result = get_tecs()

In [4]:
def add_tacs(list, index):
    for tec in result:
        list.append(tec.iloc[index])
    return list

In [5]:
parsed = lambda list, price, amount, index: add_tacs(list, index)

In [6]:
path = "./" 

import data_util
from data_generator import DataGenerator

def load_datasets(dirs):
  sets = []  
  for raw_dir in dirs:
    data_gen = DataGenerator(random=False, first_index=10, base_dir = raw_dir)
    data_gen.rewind()
    full = data_gen.steps - 2000
    print("Loading {} from dir {}".format(full, raw_dir))
    sets.append(data_util.get_sets(data_gen, full, val_percentage = 0.003, path = path, use_cache=False, on_state_parsed = parsed))
  return sets

def conc_sets(sets):
  trainX = sets[0][0]
  trainY = sets[0][1]
  valX = sets[0][2]
  valY = sets[0][3]
  for i in range(1,  len(sets)):
    data_set = sets[i]
    trainX = np.append(data_set[0], trainX, axis = 0)
    trainY = np.append(data_set[1], trainY, axis = 0)
    valX = np.append(data_set[2], valX, axis = 0)
    valY = np.append(data_set[3], valY, axis = 0)
  return trainX, trainY, valX, valY

sets = load_datasets(["stock_data/"])
trainX, trainY, valX, valY = conc_sets(sets)
np.save(path + "train" + "X.npy", trainX)
np.save(path + "train" + "Y.npy", trainY)
np.save(path + "val" + "X.npy", valX)
np.save(path + "val" + "Y.npy", valY)



Loading 886156 from dir stock_data/



(883497, 33)
(883497,)
(2658, 33)
(2658,)


In [ ]:
next = data_gen.next()

In [13]:
valX[-1][20:]

array([ 1.28085704e+00,  1.00000000e+00,  9.40000000e+01,  0.00000000e+00,
        9.25270572e-03,  5.60559789e-02,  0.00000000e+00,  0.00000000e+00,
        3.88625900e-01,  2.24289525e-01, -8.15525325e-01,  3.62931304e+00,
        1.19209233e-05])